In [1]:
import cv2
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import math
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

# eye movement tracking data

In [2]:
emt_data = pd.read_csv("ccd_100test_videos Data Export_0705_p2_1.csv")
emt_data = emt_data[emt_data['Participant name']== "Participant_2"]
emt_data = emt_data[emt_data['Eye movement type'].isin(["Fixation","Saccade","Unclassified"])]
emt_data = emt_data[emt_data['Presented Media name'].notnull()]
emt_data = emt_data[emt_data['Presented Media name'].str.contains('Text') == False]
# first_index_of_fixation = emt_data.loc[emt_data.first_valid_index(),'Eye movement type index']
# emt_data['Eye movement type index'] = emt_data['Eye movement type index'] - first_index_of_fixation

## Gaze

In [3]:
gaze_data = pd.DataFrame(emt_data, columns = ['Eyetracker timestamp [μs]','Gaze point X [DACS px]', 'Gaze point Y [DACS px]','Gaze point X [MCS norm]',
       'Gaze point Y [MCS norm]','Presented Media name', 'Original Media width [px]','Original Media height [px]', 'Eye movement type', 'Gaze event duration [ms]',
        'Eye movement type index'])
gaze_data = gaze_data.reset_index(drop = True)
gaze_data

,Eyetracker timestamp [μs],Gaze point X [DACS px],Gaze point Y [DACS px],Gaze point X [MCS norm],Gaze point Y [MCS norm],Presented Media name,Original Media width [px],Original Media height [px],Eye movement type,Gaze event duration [ms],Eye movement type index
0,2.222180e+12,922.0,486.0,0.4802,0.4496,000012_positive__frame_0.jpg,1280.0,720.0,Fixation,617.0,88.0
1,2.222180e+12,928.0,495.0,0.4832,0.4582,000012_positive__frame_0.jpg,1280.0,720.0,Fixation,617.0,88.0
2,2.222180e+12,928.0,489.0,0.4835,0.4532,000012_positive__frame_0.jpg,1280.0,720.0,Fixation,617.0,88.0
3,2.222180e+12,921.0,479.0,0.4795,0.4437,000012_positive__frame_0.jpg,1280.0,720.0,Fixation,617.0,88.0
4,2.222180e+12,920.0,495.0,0.4791,0.4582,000012_positive__frame_0.jpg,1280.0,720.0,Fixation,617.0,88.0
...,...,...,...,...,...,...,...,...,...,...,...
59753,2.222810e+12,579.0,607.0,0.3016,0.5622,002999_negative__frame_49.jpg,1280.0,720.0,Fixation,1075.0,1441.0
59754,2.222810e+12,581.0,614.0,0.3024,0.5687,002999_negative__frame_49.jpg,1280.0,720.0,Fixation,1075.0,1441.0
59755,2.222810e+12,576.0,609.0,0.3002,0.5635,002999_negative__frame_49.jpg,1280.0,720.0,Fixation,1075.0,1441.0
59756,2.222810e+12,581.0,618.0,0.3024,0.5726,002999_negative__frame_49.jpg,1280.0,720.0,Fixation,1075.0,1441.0


# video list

In [4]:
video_list = gaze_data['Presented Media name'].apply(lambda x: x[0:8]).unique()
video_list

array(['000012_p', '000015_p', '000021_n', '000029_p', '000050_p',
       '000057_p', '000075_p', '000106_p', '000152_p', '000210_n',
       '000217_p', '000254_p', '000257_p', '000258_n', '000275_p',
       '000311_p', '000313_n', '000313_p', '000340_p', '000344_p',
       '000353_p', '000360_p', '000369_p', '000385_p', '000394_p',
       '000396_n', '000401_p', '000418_p', '000425_p', '000427_p',
       '000430_n', '000480_n', '000488_p', '000494_p', '000539_n',
       '000607_n', '000649_p', '000650_n', '000709_p', '000713_n',
       '000714_p', '000716_n', '000746_p', '000770_n', '000782_n',
       '000796_p', '000838_n', '000849_p', '000858_p', '000882_p',
       '000961_p', '000966_n', '000998_n', '001004_n', '001091_p',
       '001094_p', '001096_p', '001107_p', '001190_n', '001196_n',
       '001209_n', '001261_p', '001276_n', '001301_p', '001313_p',
       '001320_n', '001383_p', '001390_p', '001395_n', '001395_p',
       '001399_n', '001420_n', '001434_p', '001450_n', '001451

# Risk objects data

In [5]:
RiskObj_Feature = pd.read_csv("RiskObj_Feature.csv")[['video','frame','Risk_Obj_feature']]
RiskObj_Feature.head()

,video,frame,Risk_Obj_feature
0,353,0,"{""region_id"": {""0"": 0, ""1"": 1}, ""x1"": {""0"": 64..."
1,353,1,"{""region_id"": {""0"": 0, ""1"": 1}, ""x1"": {""0"": 64..."
2,353,2,"{""region_id"": {""0"": 0, ""1"": 1}, ""x1"": {""0"": 64..."
3,353,3,"{""region_id"": {""0"": 0, ""1"": 1}, ""x1"": {""0"": 63..."
4,353,4,"{""region_id"": {""0"": 0, ""1"": 1}, ""x1"": {""0"": 63..."


In [6]:
# draw the gaze on frames within AOI of risk objs

for video in video_list:
    
    video_No = video[0:6]
    if video[-1] == 'p':
        class_name = 'positive'
        folder = 'frames_RiskObj/'
        tail = '_riskobj.jpg'
    else:
        class_name = 'negative'
        folder = '50_negatives/'
        tail = '.jpg'
    
    for frameindex in list(range(0,50)):

        image = Image.open(folder+ video_No + "_Frame_" + str(frameindex) + tail)
        draw = ImageDraw.Draw(image)
    
        # draw gazes
        if(gaze_data[gaze_data['Presented Media name'] == video_No + "_" + class_name + "__frame_" + str(frameindex) + ".jpg"].empty is False):
            width = gaze_data[gaze_data['Presented Media name'] == video_No + "_" + class_name + "__frame_" + str(frameindex) + ".jpg"]['Original Media width [px]'].values[0]
            height = gaze_data[gaze_data['Presented Media name'] == video_No + "_" + class_name + "__frame_" + str(frameindex) + ".jpg"]['Original Media height [px]'].values[0]

            attention_x = gaze_data[gaze_data['Presented Media name'] == video_No + "_" + class_name + "__frame_" + str(frameindex) + ".jpg"]['Gaze point X [MCS norm]']
            attention_y = gaze_data[gaze_data['Presented Media name'] == video_No + "_" + class_name + "__frame_" + str(frameindex) + ".jpg"]['Gaze point Y [MCS norm]']

            attention_x = attention_x.tolist()
            attention_y = attention_y.tolist()

            for a in list(range(0,len(attention_x))):
                if gaze_data[gaze_data['Presented Media name'] == video_No + "_" + class_name + "__frame_" + str(frameindex) + ".jpg"]['Eye movement type'].values[a]=='Fixation':
                    draw.ellipse((attention_x[a]*width-3, attention_y[a]*height-3, attention_x[a]*width+3,attention_y[a]*height+3), fill="red")
                elif(gaze_data[gaze_data['Presented Media name'] == video_No + "_" + class_name + "__frame_" + str(frameindex) + ".jpg"]['Eye movement type'].values[a]=='Saccade'):
                    draw.ellipse((attention_x[a]*width-3, attention_y[a]*height-3, attention_x[a]*width+3,attention_y[a]*height+3), fill="lime")
                elif(gaze_data[gaze_data['Presented Media name'] == video_No + "_" + class_name + "__frame_" + str(frameindex) + ".jpg"]['Eye movement type'].values[a]=='Unclassified'):
                    draw.ellipse((attention_x[a]*width-3, attention_y[a]*height-3, attention_x[a]*width+3,attention_y[a]*height+3), fill="blue")

    # display(image)

                #save image within fixation and boxes
        image.save("gaze_on_frames(p2_1)/" +  video_No + "_" + class_name + "_frame_" + str(frameindex) + tail)
        
